In [1]:
using LinearAlgebra

In [2]:
g = 9.81
ℓ = 1.0
J = 1.0

1.0

In [3]:
#Nonlinear dynamics
function dynamics(x,u)
    θ = x[1]
    θ̇ = x[2]
    
    θ̈ = -(g/ℓ)*sin(θ) + u/J
    
    return [θ̇; θ̈]
end

dynamics (generic function with 1 method)

In [4]:
#x = [θ θ̇ θ^2 θ*θ̇ θ̇^2 θ^3 θ^2*θ̇ θ*θ̇^2 θ̇^3] #extended state vector
A = [0 1 0 0 0 0 0 0 0;
     -g/ℓ 0 0 0 0 g/(6*ℓ) 0 0 0;
    0 0 0 2 0 0 0 0 0;
    0 0 -g/ℓ 0 1 0 0 0 0;
    0 0 0 -2*g/ℓ 0 0 0 0 0;
    0 0 0 0 0 0 3 0 0;
    0 0 0 0 0 -g/ℓ 0 2 0;
    0 0 0 0 0 0 -2*g/ℓ 0 1;
    0 0 0 0 0 0 0 -3*g/ℓ 0];

B0 = [0; 1.0/J; 0; 0; 0; 0; 0; 0; 0]

B = [zeros(3,9);
     1.0/J 0 0 0 0 0 0 0 0;
     0 1.0/J 0 0 0 0 0 0 0;
     zeros(1,9);
     0 0 1.0/J 0 0 0 0 0 0;
     0 0 0 2.0/J 0 0 0 0 0;
     0 0 0 0 3.0/J 0 0 0 0];

C = [I(2) zeros(2,7)]

2×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 2 stored entries:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 

In [5]:
function extend(x)
    θ = x[1]
    θ̇ = x[2]
    
    x̃ = [θ; θ̇; θ^2; θ*θ̇; θ̇^2; θ^3; θ^2*θ̇; θ*θ̇^2; θ̇^3] #extended state vector
end

extend (generic function with 1 method)

In [6]:
function dynamics3(x,u)    
    x̃ = extend(x)
    
    x̃̇ = A*x̃ + B0*u + u*(B*x̃)
    
    return C*x̃̇
end 

function dynamics1(x,u)
    θ = x[1]
    θ̇ = x[2]
    
    θ̈ = -(g/ℓ)*θ + u/J
    
    return [θ̇; θ̈]
end 

dynamics1 (generic function with 1 method)

In [7]:
#The original 2D state is controllable at the origin, but not the extended state
CRB = [B0 A*B0 A*A*B0 A*A*A*B0 A*A*A*A*B0 A*A*A*A*A*B0 A*A*A*A*A*A*B0 A*A*A*A*A*A*A*B0 A*A*A*A*A*A*A*A*B0]
rank(CRB)

2

In [18]:
xsamp = randn(2)
usamp = 0.0 #randn()

0.0

In [19]:
#Extended state is controllable as long as you're away from the origin
B̃ = B0 + B*extend(xsamp)
CRB = [B̃ A*B̃ A*A*B̃ A*A*A*B̃ A*A*A*A*B̃ A*A*A*A*A*B̃ A*A*A*A*A*A*B̃ A*A*A*A*A*A*A*B̃ A*A*A*A*A*A*A*A*B̃]
rank(CRB)

9

In [20]:
dynamics(xsamp, usamp)

2-element Vector{Float64}:
 -0.8032568824704643
 -9.164331170088818

In [21]:
dynamics3(xsamp, usamp)

2-element Vector{Float64}:
 -0.8032568824704643
 -7.131245295163822

In [22]:
dynamics1(xsamp, usamp)

2-element Vector{Float64}:
  -0.8032568824704643
 -18.988544936305313

In [23]:
xsamp

2-element Vector{Float64}:
  1.9356314919781155
 -0.8032568824704643